In [207]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import pickle as pkl
from time import sleep
import random
from fake_useragent import UserAgent
from collections import Iterable


<ipython-input-207-d7cdcd16b49c>:8: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import Iterable


In [137]:
ua = UserAgent()
user_agent = {'User-agent': ua.random}
print(user_agent)

{'User-agent': 'Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML like Gecko) Chrome/44.0.2403.155 Safari/537.36'}


In [227]:
ua = UserAgent()
user_agent = {'User-agent': ua.random}
print(user_agent)
sleep(random.randint(3, 7))
response = requests.get('https://topdocumentaryfilms.com/watch-online/', headers = user_agent)
#response
soup = BeautifulSoup(response.text, 'html5lib')
main = soup.find(class_='module')
categories = main.find_all('div', class_='sitemap-wraper clear')
cat_list = []
links = []
num_in_cat = []
for i in categories:
    cat = i.find('a').text
    cat_list.append(cat)
    link = i.find('a').get('href')
    links.append(link)
print(cat_list, links)

{'User-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:10.0) Gecko/20100101 Firefox/62.0'}
['Technology', 'Sports', 'Society', 'Sexuality', 'Science', 'Religion', 'Psychology', 'Politics', 'Philosophy', 'Performing Arts', 'Nature', 'Mystery', 'Military and War', 'Media', 'History', 'Health', 'Environment', 'Economics', 'Drugs', 'Crime', 'Conspiracy', 'Biography', 'Art and Artists', '9/11'] ['https://topdocumentaryfilms.com/category/technology/', 'https://topdocumentaryfilms.com/category/sports/', 'https://topdocumentaryfilms.com/category/society/', 'https://topdocumentaryfilms.com/category/sex/', 'https://topdocumentaryfilms.com/category/science-technology/', 'https://topdocumentaryfilms.com/category/religion/', 'https://topdocumentaryfilms.com/category/psychology/', 'https://topdocumentaryfilms.com/category/politics/', 'https://topdocumentaryfilms.com/category/philosophy/', 'https://topdocumentaryfilms.com/category/music-performing-arts/', 'https://topdocumentaryfilms.com/ca

In [223]:
def flatten(lst):
    rt = []
    for i in lst:
        if isinstance(i, list): rt.extend(flatten(i))
        else:rt.append(i)
    return rt

links_to_click = []
for link in links:
    temp = []
    i=1
    url = link
    links_to_click.append(url)
    while True:
        i = i+1
        page = requests.get(url, headers = user_agent)
        if page.status_code != 200:
            break
        url = link + "page/" + str(i) + "/"
        temp.append(url)
        cleaned = temp[0:(len(temp)-1)]
    links_to_click.append(cleaned)
final_list = flatten(links_to_click)

In [371]:
titles = []
categories = []
years = []
ratings = []
for item in final_list:
    source_code = requests.get(item, headers = user_agent)
    soup = BeautifulSoup(source_code.text, 'html5lib')
    main = soup.find(class_='group grid')
    result = main.find_all('h2')
    for i in result:
        try:
            title = i.text
            titles.append(title)
        except AttributeError:
            titles.append('NaN')
    result2 = main.find_all('article', class_='module')
    for i in result2:
        year_cat = i.find('div', class_='meta-bar').contents
        try:
            year = year_cat[0].strip('\n')
            final_year = year.strip(',\t')
            years.append(final_year)
        except AttributeError:
            years.append('NaN')
        try:
            cat = i.find(rel = 'category tag').text
            categories.append(cat)
        except AttributeError:
            categories.append('NaN')
    result3 = main.find_all(class_='archive_rating')
    for i in result3:
        try:
            rating = i.text.strip('★ ')
            ratings.append(rating)
        except AttributeError:
            ratings.append('NaN')
        



In [373]:
dictionary = dict({'Title':titles, 'User Rating - Top Doc':ratings, 'Years': years, 'Category':categories})
df = pd.DataFrame(dictionary)
df

,Title,User Rating - Top Doc,Years,Category
0,Printing Out the World,7.77,2019,Technology
1,The Big Reset 2.0,7.33,2020,Technology
2,AlphaGo,8.06,2017,Technology
3,Charlie and the Humans,7.38,2019,Technology
4,Into the Unknown,8.40,2016,Technology
...,...,...,...,...
2544,The Ultimate Con,8.68,2007,9/11
2545,9/11 Mysteries,8.10,2006,9/11
2546,Towers of Deception: The Media Cover-Up of 9/11,7.78,2003,9/11
2547,In Their Own Words: The Untold Stories of the ...,7.64,2006,9/11


In [374]:
df.to_pickle('top_doc.pkl')

______

In [229]:
main = soup.find(class_='group grid')
result = main.find_all(class_='excerptpic')
#to get the title
just_one.find('a').get('title')

'Printing Out the World'

In [346]:
result2 = main.find_all('article', class_='module')
year_cat = result2[0].find('div', class_='meta-bar').contents
year = year_cat[0].strip('\n')
final_year = year.strip(',\t')
#cat = result2[0].find(')
#year_cat = result2[0].text.split(',')
#year = year_cat[0].strip('\n').split('\n\n')[1]
#cat = year_cat[1].strip('\t').split('  \n')[0]
cat = result2[0].find(rel = 'category tag').text
cat

'Technology'

In [153]:
result3 = main.find_all(class_='archive_rating')
rating = result3[0].text.strip('★ ')
rating

'7.77'

In [95]:
print(list_links[1:4]) #use pagination module at the end of the page to find the stop

['https://topdocumentaryfilms.com/category/sports/page/2/', 'https://topdocumentaryfilms.com/category/sports/page/3/', 'https://topdocumentaryfilms.com/category/sports/page/4/']


In [168]:
result4 = main.find(class_='pagination module')
result4

<div class="pagination module"><span class="current">1</span><a class="inactive" href="https://topdocumentaryfilms.com/category/technology/page/2/">2</a>... <a href="https://topdocumentaryfilms.com/category/technology/page/10/">10</a><a href="https://topdocumentaryfilms.com/category/technology/page/2/">Next</a></div>

In [169]:
next_=result4.find_all('a')[2].get('href')

In [170]:
next_

'https://topdocumentaryfilms.com/category/technology/page/2/'

In [149]:
'''
#to get the links of each subsequent page 
i=2
links_in_cat = []
temp = []
links_in_cat.append('https://topdocumentaryfilms.com/category/sports/')
print(links_in_cat)

while True:
    i = i+1
    page = requests.get(url, headers = user_agent)
    if page.status_code != 200:
        break
    url = 'https://topdocumentaryfilms.com/category/sports/' + "page/" + str(i) + "/"
    temp.append(url)
cleaned = temp[1:(len(temp)-1)]
links_in_cat.append(cleaned)
print(links_in_cat)

['https://topdocumentaryfilms.com/category/sports/']
['https://topdocumentaryfilms.com/category/sports/', []]


In [212]:
"""
def flatten(coll):
    for i in coll:
        if isinstance(i, Iterable) and not isinstance(i, basestring):
            for subc in flatten(i):
                yield subc
        else:
            yield i

In [209]:
'''
links_to_click = []
temp = []
i=1
url = 'https://topdocumentaryfilms.com/category/sports/'
links_to_click.append(url)
while True:
    i = i+1
    page = requests.get(url, headers = user_agent)
    if page.status_code != 200:
        break
    url = 'https://topdocumentaryfilms.com/category/sports/' + "page/" + str(i) + "/"
    temp.append(url)
    cleaned = temp[0:(len(temp)-1)]
links_to_click.append(cleaned)


In [294]:
'''
titles = []
categories = []
years = []
ratings = []
for item in final_list[:2]:
    source_code = requests.get(item, headers = user_agent)
    soup = BeautifulSoup(source_code.text, 'html5lib')
    main = soup.find(class_='group grid')
    result = main.find_all(class_='excerptpic')
    for i in result:
        try:
            title = i.find('a').get('title')
            titles.append(title)
        except AttributeError:
            titles.append('NaN')
    result2 = main.find_all('article', class_='module')
    for i in result2:
        year_cat = i.find('div', class_='meta-bar')
        year_cat = i.text.split(',')
        try:
            year = year_cat[0].strip('\n').split('\n\n')[1]
            years.append(year)
        except AttributeError:
            titles.append('NaN')
        try:
            cat = year_cat[1].strip('\t').split('  \n')[0]
            categories.append(cat)
        except AttributeError:
            categories.append('NaN')
    result3 = main.find_all(class_='archive_rating')
    for i in result3:
        try:
            rating = i.text.strip('★ ')
            ratings.append(rating)
        except AttributeError:
            ratings.append('NaN')
        


IndexError: list index out of range

In [372]:
len(titles)

2549

In [369]:
titles = []
for item in final_list[7:9]:
    source_code = requests.get(item, headers = user_agent)
    soup = BeautifulSoup(source_code.text, 'html5lib')
    main = soup.find(class_='group grid')
    result = main.find_all('h2')
    for i in result:
        title = i.text
        titles.append(title)
print(titles)
    #for i in result:
    #    title = 
#    for i in result:
#        try:
#            title = i.find('a').get('title')
#            titles.append(title)
#        except AttributeError:
#            titles.append('NaN')

['Playing God', 'In the Realm of the Hackers', 'Build It Bigger: Floating City', 'Tetris: From Russia with Love', 'Future Intelligence', 'Can You Hack It? - Hackers Wanted', 'A Machine to Die For: The Quest for Free Energy', 'The Betrayal by Technology: A Portrait of Jacques Ellul', 'Inventions That Changed the World', 'Future by Design', 'Revolution OS', 'The Virtual Revolution', 'Dubai Palm Islands', 'World Trade Center', 'James Burke: Connections', 'The Machine That Made Us', 'Download: The True Story of the Internet', 'Carrier', 'Battle of the X-Planes', 'Car of the Future', 'Nerds 2.0.1: A Brief History of the Internet', 'Triumph of the Nerds: The Rise of Accidental Empires', 'Trinity and Beyond: The Atomic Bomb Movie', 'The Machine That Changed the World']
